In [17]:
import json
import re
import pandas as pd

# =================配置区域=================
INPUT_FILE = "results.jsonl"  # 你的结果文件路径
# =========================================

def extract_model_answer(text):
    """
    从模型的输出中提取 A 或 B。
    策略：找 "Answer:" 后面的第一个 A 或 B，或者找最后出现的 A/B。
    """
    if not isinstance(text, str):
        return "Unknown"
    
    # 优先匹配明确的格式 "Answer: A" 或 "Answer: <A>"
    match = re.search(r"Answer:\s*[:<]?\s*([AB])\b", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    # 如果没找到标准格式，尝试找文本最后出现的选项（这是一个兜底策略）
    # 也就是寻找最后出现的 "A" 或 "B" 字符
    matches = re.findall(r"\b([AB])\b", text)
    if matches:
        return matches[-1].upper()
    
    return "Unknown"

def detect_question_type(item):
    """
    根据选项文本内容判断是 NT 还是 NF。
    逻辑依据：
    - NT (Negation True): 正确选项包含否定词。
    - NF (Negation False): 错误选项包含否定词。
    """
    ground_truth = item.get("ground_truth", "").strip().upper()
    inference_A = item.get("inference_A", "")
    inference_B = item.get("inference_B", "")
    
    if ground_truth not in ["A", "B"]:
        return "Error"

    # 1. 确定哪个文本是“正确选项”，哪个是“错误选项”
    if ground_truth == "A":
        correct_text = inference_A
        incorrect_text = inference_B
    else:
        correct_text = inference_B
        incorrect_text = inference_A
        
    # 2. 定义否定词列表 (根据 VIBE 论文逻辑)
    negation_keywords = ["not", "don't", "doesn't", "didn't", "cannot", "never"]
    
    def has_negation(text):
        text_lower = text.lower()
        return any(kw in text_lower for kw in negation_keywords)

    is_correct_negated = has_negation(correct_text)
    is_incorrect_negated = has_negation(incorrect_text)

    # 3. 分类逻辑
    if is_correct_negated:
        return "NT"  # 正确答案是否定的 -> Negation True
    elif is_incorrect_negated:
        return "NF"  # 错误答案是否定的 -> Negation False
    else:
        return "Standard" # 其他类型（论文中的 Fixed/Distractor 若无否定词通常落入此类）

def evaluate_vibe(file_path):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip(): continue
            item = json.loads(line)
            
            # 1. 提取预测结果
            prediction = extract_model_answer(item.get("model_output"))
            truth = item.get("ground_truth")
            
            # 2. 判断正误
            is_correct = (prediction == truth)
            
            # 3. 判断题目类型
            q_type = detect_question_type(item)
            
            data.append({
                "Task ID": item.get("task_id"),
                "Type": q_type,
                "Correct": is_correct,
                "Model Output": prediction,
                "Ground Truth": truth
            })
            
    df = pd.DataFrame(data)
    
    # 计算各类别准确率
    results = {}
    
    # 计算 Overall (All)
    results['All'] = df['Correct'].mean() * 100
    
    # 计算各子类别 (NT, NF, Standard)
    for q_type in ['NT', 'NF', 'Standard']:
        subset = df[df['Type'] == q_type]
        if len(subset) > 0:
            results[q_type] = subset['Correct'].mean() * 100
        else:
            results[q_type] = 0.0
            
    # 格式化输出表格
    print("\n" + "="*50)
    print("VIBE Evaluation Results (Accuracy %)")
    print("="*50)
    
    # 创建类似论文的表格 DataFrame
    # 注意：这里用 'Standard' 代替了 Fixed/Distractor，因为那是内部元数据
    # 如果你想区分 Fixed/Distractor，需要在 jsonl 里有额外的 tag 字段
    
    summary_df = pd.DataFrame([{
        "Model": "Qwen2.5-VL-7B",
        "Standard": f"{results['Standard']:.1f}",
        "NF": f"{results['NF']:.1f}",
        "NT": f"{results['NT']:.1f}",
        "All": f"{results['All']:.1f}"
    }])
    # summary_df = pd.DataFrame([{
    #         "Model": "Qwen2.5-VL-3B",
    #         "Standard": f"70.1",
    #         "NF": f"81.1",
    #         "NT": f"30.6",
    #         "All": f"68.6"
    #     }])
    
    print(summary_df.to_string(index=False))
    print("="*50)
    
    # 打印详细统计
    print(f"\n样本统计:\nTotal: {len(df)}\nNT count: {len(df[df['Type']=='NT'])}\nNF count: {len(df[df['Type']=='NF'])}\nStandard count: {len(df[df['Type']=='Standard'])}")

    return df

if __name__ == "__main__":
    # 请确保你的 results.jsonl 文件里包含了 inference_A 和 inference_B
    # 如果没有，你需要先用代码把原始数据 merge 进去
    if "inference_A" not in open(INPUT_FILE).readline():
        print("错误：JSONL 文件中缺少 'inference_A/B' 字段。无法进行 NT/NF 分类。")
        print("请修改之前的脚本，在写入 results.jsonl 时把原始 item 的所有字段都写进去。")
    else:
        df = evaluate_vibe(INPUT_FILE)


VIBE Evaluation Results (Accuracy %)
        Model Standard   NF   NT  All
Qwen2.5-VL-7B     66.9 76.4 35.5 66.1

样本统计:
Total: 987
NT count: 138
NF count: 368
Standard count: 481


# Ollama test

In [1]:
import cv2
import ollama
from PIL import Image
import io

# ================= 配置区域 =================
# 1. 你的模型名字 (必须是你 ollama list 里有的)
MODEL_NAME = "qwen2.5vl:3b"  # 或者是 "internvl2", "llava" 等

# 2. 视频路径
VIDEO_PATH = "./test2.mp4" # 替换为你的视频文件路径

# 3. 抽帧策略 (关键！)
# 如果视频长 10 秒，SAMPLES=10 代表每秒截一张
TARGET_FRAME_COUNT = 15 
# ===========================================

def extract_frames(video_path, target_count=8):
    """
    从视频中均匀抽取指定数量的帧，并转换为字节流
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    duration = total_frames / fps if fps > 0 else 0
    
    print(f"🎬 视频信息: 时长 {duration:.2f}s | 总帧数 {total_frames} | FPS {fps}")
    
    # 计算抽帧间隔
    step = max(1, total_frames // target_count)
    
    frame_images = [] # 存放发给 ollama 的图片数据
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # 按照步长抽帧
        if count % step == 0 and saved_count < target_count:
            # OpenCV 默认是 BGR，需要转为 RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # ⬇️ 优化：调整图片大小 (可选)
            # 为了加快速度和节省显存，可以将图片缩小，例如最大边长 720 或 512
            pil_img.thumbnail((336, 336)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=80)
            image_bytes = img_byte_arr.getvalue()
            
            frame_images.append(image_bytes)
            saved_count += 1
            
        count += 1

    cap.release()
    print(f"✅ 成功抽取 {len(frame_images)} 帧用于分析")
    return frame_images

def analyze_video(video_path):
    # 1. 抽取视频帧
    frames = extract_frames(video_path, target_count=TARGET_FRAME_COUNT)
    
    if not frames:
        return

    # 2. 构建 Prompt
    prompt = "请仔细观看这一组连续的视频帧，详细描述视频里发生了什么？包括人物的动作、表情和环境变化。"

    print("🤖 模型正在推理中 (可能需要几秒到几分钟，取决于显卡)...")
    
    # 3. 发送给 Ollama
    # 关键点：'images' 参数接受一个列表，我们把所有抽出来的帧都放进去
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{
                'role': 'user',
                'content': prompt,
                'images': frames  # 这里传入所有帧
            }]
        )
        
        print("\n" + "="*20 + " 识别结果 " + "="*20)
        print(response['message']['content'])
        print("="*50)
        
    except Exception as e:
        print(f"❌ 推理出错: {e}")
        print("提示: 请检查模型是否支持多图片输入 (Vision Model)，或显存是否不足。")

if __name__ == "__main__":
    analyze_video(VIDEO_PATH)

🎬 视频信息: 时长 2.35s | 总帧数 54 | FPS 23
✅ 成功抽取 15 帧用于分析
🤖 模型正在推理中 (可能需要几秒到几分钟，取决于显卡)...

==================== 识别结果 ====================
视频中，两位男性角色在一个室内环境中进行对话。左侧的男性穿着蓝色衬衫，背对着镜头，而右侧的男性则穿着深蓝色长袖上衣，面对镜头。两人似乎正在进行一场严肃的对话，表情认真，时而点头，时而皱眉，显示出讨论的激烈程度。

背景中，可以看到一个棕色的柜子，上面摆放着一个红色的玩具车。柜子旁边有一张桌子，上面放着一个黑色的物体，可能是装饰品。房间的墙壁是浅色的，营造出一种温馨的氛围。整个场景给人一种家庭或朋友间的亲密对话的感觉。


# Ollama

In [3]:
import json
import os
import cv2
import io
import ollama
from PIL import Image

# ================= 配置区域 =================
# 你的 Ollama 模型名字
MODEL_NAME = "qwen2.5vl:3b"  # 确保你 ollama pull 了这个模型

# 视频文件夹的路径
VIDEO_FOLDER = "./videos"
INPUT_FILE = "vibe_task.jsonl"
OUTPUT_FILE = "results.jsonl"

# 抽帧数量：根据显存大小调整。
# 7B 模型在 Mac 上建议 8-16 帧，帧数越多越卡，但细节越好。
TARGET_FRAME_COUNT = 8 
# ===========================================

def extract_frames(video_path, target_count=8):
    """
    读取视频文件，均匀抽取指定数量的帧，并转为字节流列表
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 简单的均匀抽帧逻辑
    step = max(1, total_frames // target_count)
    frame_images = []
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        if count % step == 0 and saved_count < target_count:
            # OpenCV BGR -> RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 缩放图片以节省显存 (可选，建议长边不超过 768 或 512)
            # Qwen2.5-VL 支持任意分辨率，但太大会导致推理慢
            pil_img.thumbnail((512, 512)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=80)
            frame_images.append(img_byte_arr.getvalue())
            saved_count += 1
            
        count += 1

    cap.release()
    return frame_images

# def process_data():
    # 检查输出文件是否存在，如果存在，统计行数以便断点续传（可选）
processed_ids = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    processed_ids.add(data.get("task_id"))
                except:
                    pass

with open(INPUT_FILE, 'r', encoding='utf-8') as f_in, \
        open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out: # 使用 'a' 模式追加
    
    for line_number, line in enumerate(f_in):
        if not line.strip(): continue
        
        item = json.loads(line)
        task_id = item.get("task_id")
        
        # 简单的断点续传逻辑
        if task_id in processed_ids:
            print(f"Skipping processed task: {task_id}")
            continue

        video_filename = item.get("video")
        fact_text = item.get("fact")
        inference_A = item.get("inference_A")
        inference_B = item.get("inference_B")
        
        # --- 步骤 1: 获取视频绝对路径 ---
        abs_video_path = os.path.abspath(os.path.join(VIDEO_FOLDER, video_filename))
        if not os.path.exists(abs_video_path):
            print(f"Error: Video file not found: {abs_video_path}")
            continue
        
        # --- 步骤 2: (核心修改) 抽取视频帧 ---
        # 本地 Ollama 不支持直接传文件路径，必须传图片列表
        print(f"Processing Task: {task_id} | Extracting frames from {video_filename}...")
        frames = extract_frames(abs_video_path, target_count=TARGET_FRAME_COUNT)
        
        if not frames:
            print(f"Error: No frames extracted for {task_id}")
            continue

        # --- 步骤 3: 构建 Prompt ---
        text_prompt = (     
            f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
            f"You will be given a set of video frames representing a clip with one main speaker. " # 微调文案
            f"Follow the Task Guidelines and the Response Format.\n"            
            f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
            f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
            f"- Finally, give your answer according to the Response Format.\n\n"
            f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
            f"Answer: <A OR B>\n\n"
            f"### Fact\n{fact_text}\n\n"
            f"### Interpretations\n"
            f"A. {inference_A}\n"
            f"B. {inference_B}\n\n"
            f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
        )

        try:
            # --- 步骤 4: (核心修改) 调用 Ollama ---
            response = ollama.chat(
                model=MODEL_NAME,
                messages=[{
                    'role': 'user',
                    'content': text_prompt,
                    'images': frames  # 关键：传入抽取的帧数据列表
                }],
                options={
                    'temperature': 0.7,
                    # 'num_ctx': 4096 # 如果报错上下文不足，可以调大这个
                }
            )
            
            response_content = response['message']['content']
            print(f"✅ Result for {task_id} generated.")

            # 保存结果
            result = {
                "task_id": task_id,
                "model_output": response_content,
                "ground_truth": item.get("answer"),
                "inference_A": inference_A, 
                "inference_B": inference_B
            }
            
            f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
            f_out.flush()
            
        except Exception as e:
            print(f"❌ Error processing {task_id}: {e}")

Processing Task: video_1 | Extracting frames from video_1.mp4...
✅ Result for video_1 generated.
Processing Task: video_1 | Extracting frames from video_1.mp4...
✅ Result for video_1 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_2 | Extracting frames from video_2.mp4...
✅ Result for video_2 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_4 | Extracting frames from video_4.mp4...
✅ Result for video_4 generated.
Processing Task: video_5 | Ext

# Output process

In [ ]:
import json
import re
import pandas as pd

# ... (配置区域保持不变) ...

def extract_model_answer_strict(text):
    """
    严格提取 A 或 B。
    """
    if not isinstance(text, str):
        return None
    
    text = text.strip()
    
    # 0. 极简情况：模型只输出了 "A" 或 "B" (或带点)
    if text in ["A", "B", "A.", "B."]:
        return text[0]

    # 1. 匹配标准 "Answer: A" 格式 (兼容各类括号)
    match = re.search(r"Answer\s*:?\s*[\(<\[\{]?\s*([AB])\b", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    # 2. 匹配句尾明确的选项
    match_sentence = re.search(r"(?:answer|choice|option)\s*(?:is|:)\s*([AB])\b", text, re.IGNORECASE)
    if match_sentence:
        return match_sentence.group(1).upper()
    
    # 3. 匹配文本末尾的 A 或 B (确保前面有空格或标点)
    match_end = re.search(r"[\s\.,]([AB])[\.\)]?$", text)
    if match_end:
        return match_end.group(1).upper()

    return None

def detect_question_type(item):
    """
    判断题目类型 (NT/NF/Standard)
    修复了否定词匹配逻辑
    """
    ground_truth = item.get("ground_truth", "").strip().upper()
    inference_A = item.get("inference_A", "")
    inference_B = item.get("inference_B", "")
    
    if ground_truth not in ["A", "B"]:
        return "Error"

    # 确定正确选项文本和错误选项文本
    if ground_truth == "A":
        correct_text = inference_A
        incorrect_text = inference_B
    else:
        correct_text = inference_B
        incorrect_text = inference_A
        
    # === 修改点：更全面的否定判定逻辑 ===
    # 使用正则表达式匹配单词边界，防止 "no" 匹配到 "know"
    # 包含常见否定词：not, no, never, cannot, neither, nor
    # 以及缩写形式：n't (don't, can't, won't 等)
    negation_pattern = r"\b(not|no|never|cannot|neither|nor)\b|n't\b"
    
    def has_negation(text):
        # 搜索是否存在符合正则的否定词
        return bool(re.search(negation_pattern, text, re.IGNORECASE))

    is_correct_negated = has_negation(correct_text)
    is_incorrect_negated = has_negation(incorrect_text)

    # 逻辑判断：
    # NT (Negation True): 正确答案包含否定
    # NF (Negation False): 错误答案包含否定
    if is_correct_negated:
        return "NT"
    elif is_incorrect_negated:
        return "NF"
    else:
        return "Standard"


def process_results(input_path, manual_output_path):
    valid_data = []
    manual_data = []
    
    print(f"正在读取 {input_path} ...")
    
    with open(input_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            if not line.strip(): continue
            item = json.loads(line)
            
            prediction = extract_model_answer_strict(item.get("model_output"))
            
            if prediction is None:
                item['original_line'] = line_num 
                manual_data.append(item)
            else:
                truth = item.get("ground_truth")
                is_correct = (prediction == truth)
                q_type = detect_question_type(item)
                
                valid_data.append({
                    "Task ID": item.get("task_id"),
                    "Type": q_type,
                    "Correct": is_correct, # 这里存的是 True/False
                    "Model Output": prediction,
                    "Ground Truth": truth
                })
    
    # 处理需要人工审核的数据
    if manual_data:
        print(f"\n 有 {len(manual_data)} 条数据无法自动提取，已写入 {manual_output_path}")
        with open(manual_output_path, 'w', encoding='utf-8') as f_out:
            for item in manual_data:
                f_out.write(json.dumps(item, ensure_ascii=False) + "\n")
    
    # ================= 核心统计部分 =================
    if valid_data:
        df = pd.DataFrame(valid_data)
        
        stats_list = []
        
        # 定义我们要统计的类别顺序
        categories = ['Standard', 'NF', 'NT', 'All']
        
        for cat in categories:
            if cat == 'All':
                subset = df
            else:
                subset = df[df['Type'] == cat]
            
            total_count = len(subset)
            # sum() 会把 True 当作 1，False 当作 0，所以 sum 就是正确的数量
            correct_count = subset['Correct'].sum() 
            
            acc = (correct_count / total_count * 100) if total_count > 0 else 0.0
            
            stats_list.append({
                "Category": cat,
                "Total (总数)": total_count,
                "Correct (正确数)": correct_count,
                "Accuracy (%)": f"{acc:.2f}"
            })
            
        stats_df = pd.DataFrame(stats_list)
        
        print("\n" + "="*60)
        print("VIBE 详细评估结果")
        print("="*60)
        print(stats_df.to_string(index=False))
        print("="*60)
        
    else:
        print("没有有效数据可统计。")

if __name__ == "__main__":
    process_results(INPUT_FILE, MANUAL_CHECK_FILE)

正在读取 results/qwen-7b-07.jsonl ...

VIBE 详细评估结果
Category  Total (总数)  Correct (正确数) Accuracy (%)
Standard         486            344        70.78
      NF         369            306        82.93
      NT         139             58        41.73
     All         994            708        71.23
